In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/rame/trl/examples/llama")

In [ ]:
ds["train"][0]["qid"]

In [ ]:
dtrainf

In [ ]:
from datasets import load_dataset

ds = load_dataset("lvwerra/stack-exchange-paired")

dtrainf = ds["train"].filter(lambda x: len(x["question"]) < 300, batched=False)

import numpy as np

def remove_duplicate(duplicated_dataset):
    initial_list = duplicated_dataset.map(lambda x: {"id": x['qid']})
    _ , unique_indices = np.unique(initial_list["qid"], return_index=True, axis=0)
    filtered_dataset = duplicated_dataset.select(unique_indices.tolist())
    return filtered_dataset

dtrainf_deduplicated = remove_duplicate(dtrainf)

In [ ]:
dtestf = ds["test"].filter(lambda x: len(x["question"]) < 300, batched=False)
dtestf_deduplicated = remove_duplicate(dtestf)
dtestfs = dtestf_deduplicated.select(range(1000))

In [ ]:
load_dataset("lvwerra/stack-exchange-paired", split="train", cache_dir="/data/rame/data/huggingface")

In [ ]:
split="train"

In [ ]:
import os
import numpy as np
import torch
from datasets import load_dataset, load_from_disk

from trl import PPOConfig, PPOTrainer
from trl.core import LengthSampler

import llama_utils, ppo_utils, args_utils
# see this https://github.com/lvwerra/trl/blob/main/examples/stack_llama/scripts/rl_training.py
MIN_SIZE = 5


In [ ]:
split = {"train": "train", "validation": "test"}[split]
if args_utils.LOCAL_FILES_ONLY:
    ds = load_from_disk("/gpfsdswork/projects/rech/edr/utr15kn/dataplace/data/huggingface/stack_data_sampled")
    ds = ds[split]
    # if split == "train":
    #     ds = ds.select(range(20000))
    # else:
    #     ds = ds.select(range(1000))
else:
    ds = load_from_disk("/data/rame/data/huggingface/stack_data_sampled")
    # ds = load_dataset(
    #     "lvwerra/stack-exchange-paired",
    #     cache_dir="/data/rame/data/huggingface", split=split)
    # ds = ds.select(range(20000))
    ds = ds[split]

In [ ]:
ds["question"]

In [ ]:
ds = load_dataset("lvwerra/stack-exchange-paired", split="train", cache_dir)
ds = ds.select(range(100000))

In [ ]:
ds

In [ ]:
ds = load_dataset("Anthropic/hh-rlhf", name="comparisons", split="train")

In [ ]:
from trl.core import LengthSampler

In [ ]:
import llama_utils

In [ ]:
MIN_SIZE = 5

In [ ]:
ds_filtered = ds.filter(
    lambda x: x["chosen"] is not None and MIN_SIZE <
    len(x["chosen"].split("Assistant: ")[0]) < 80,
    batched=False
)


In [ ]:
ds

In [ ]:
ds_filtered

In [ ]:
input_size_sampler = LengthSampler(2, 8)

def tokenize(sample):
    text = sample["chosen"].replace("\n", " ")
    instruction = text.split("Assistant: ")[0].split("Human: ")[1]
    prompt = llama_utils.Instructions.get_prompt_noinput(instruction=instruction,)
    response = text.split("Assistant: ")[1]
    size_prompt = len(tokenizer.encode(prompt)) - 1
    input_size = size_prompt + input_size_sampler()
    sample["input_ids"] = tokenizer.encode(prompt + response)[:input_size]
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

ds_mapped = ds_filtered.map(tokenize, batched=False, load_from_cache_file=False)
ds_mapped.set_format(type="torch")

In [ ]:
ds_mapped["query"][5]

In [ ]:
ds = ds.filter(
    lambda x: x["chosen"] is not None and MIN_SIZE < len(x) < max_size and x[
        'info']["id"] is not None,
    batched=False
)

In [ ]:
len(ds)

In [ ]:
import os
import numpy as np
import torch
from datasets import load_dataset

from trl import PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

import llama_utils, ppo_utils, args_utils

MIN_SIZE = 100


def build_dataset(dataset_name, *args, **kwargs):
    if dataset_name == "news":
        return _build_news_dataset(*args, **kwargs)
    else:
        return _build_openai_dataset(*args, **kwargs)


def _build_news_dataset(tokenizer, split="train", max_size=1500):
    """
    Args:
        dataset_name (`str`): "argilla/news-summary"
    """
    split = {"train": "test", "validation": "train"}[split]
    ds = load_dataset("argilla/news-summary", name="comparisons", split=split, use_auth_token=True)
    ds_filtered = ds.filter(
        lambda x: x["text"] is not None and MIN_SIZE < len(x["text"]) < max_size and x["id"] is
        not None,
        batched=False
    )

    def remove_duplicate(duplicated_dataset):
        initial_list = duplicated_dataset.map(lambda x: {"id": x['id']})
        _, unique_indices = np.unique(initial_list["id"], return_index=True, axis=0)
        filtered_dataset = duplicated_dataset.select(unique_indices.tolist())
        return filtered_dataset

    ds_deduplicated = remove_duplicate(ds_filtered)
    input_size_sampler = LengthSampler(2, 8)

    def tokenize(sample):
        info_post = "-".join(sample["text"].replace("\n", " ").split("(Reuters) -")[1:]).strip()
        prompt_summary = llama_utils.Instructions.get_prompt_summary(post=info_post)
        size_prompt_summary = len(tokenizer.encode(prompt_summary)) - 1
        input_size = size_prompt_summary + input_size_sampler()
        choice = 0  # select the best summary
        response = sample["prediction"][choice]["text"].replace("\n", " ").replace(".", ",")
        sample["input_ids"] = tokenizer.encode(prompt_summary + response)[:input_size]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds_mapped = ds_deduplicated.map(tokenize, batched=False, load_from_cache_file=False)
    ds_mapped.set_format(type="torch")
    return ds_mapped

In [ ]:
tokenizer = llama_utils.Tokenizer.load_tokenizer("decapoda-research/llama-7b-hf")

In [ ]:
dataset = build_dataset(tokenizer=tokenizer, dataset_name="news")

In [ ]:
print(dataset)

In [ ]:

dataset.set_format("pandas")
# df_batch = ds[:].sample(bs)
df_batch = dataset[:10]

In [ ]:
df_batch["query"][0]

In [ ]:
dataset["train"][:10]

In [ ]:
[d["query"] for d in dataset[:10]]

In [ ]:
ds = load_dataset("openai/summarize_from_feedback", name="comparisons", split="train")

In [ ]:
dsf = ds.filter(lambda x: len(x["info"]["post"]) < 1200, batched=False)

In [ ]:
dsf

In [ ]:
"\na \n aljkka \n".strip()

In [ ]:
ds["summaries"][0]

In [ ]:
load_dataset??

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("CogComp/bart-faithful-summary-detector")

article = "Ban Ki-Moon was re-elected for a second term by the UN General Assembly, unopposed and unanimously, on 21 June 2011."

bad_summary = "Ban Ki-moon was elected for a second term in 2007."
good_summary = "Ban Ki-moon was elected for a second term in 2011."

bad_pair = tokenizer(text=bad_summary, text_pair=article, return_tensors='pt')
good_pair = tokenizer(text=good_summary, text_pair=article, return_tensors='pt')

model = AutoModelForSequenceClassification.from_pretrained("CogComp/bart-faithful-summary-detector")

bad_score = model(**bad_pair)
good_score = model(**good_pair)
good_score[0][:, 1]
bad_score[0][:, 1]



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("/home/rame/trl/examples/llama")

In [ ]:
import llama_utils

In [ ]:
import torch

In [ ]:
device = 0 if torch.cuda.is_available() else "cpu"

In [ ]:
sentiment_pipe = llama_utils.Pipelines.load_pipe(sentiment_model="Tristan/gpt2_reward_summarization", device=device)

In [ ]:
sentiment_pipe_v2 = llama_utils.Pipelines.load_pipe(sentiment_model="CogComp/bart-faithful-summary-detector", device=device)

In [ ]:
texts = [
    "Ban Ki-moon was a very good president.",
    "Ban Ki-moon was elected for a second term in 2011.",
    "Zinedine Yazid Zidane, popularly known as Zizou, is a French professional football manager and former player who played as an attacking midfielder."
    
]
article = "Ban Ki-Moon was re-elected for a second term by the UN General Assembly, unopposed and unanimously, on 21 June 2011."

In [ ]:
texts_v1 = [
    llama_utils.transform_text(
        sentiment_pipe=sentiment_pipe,
        response_text=text,
        instruction=article
    ) for text in texts
]

In [ ]:
texts_v2 = [
    llama_utils.transform_text(
        sentiment_pipe=sentiment_pipe,
        response_text=text,
        instruction=article
    ) for text in texts
]

In [ ]:
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 1}

In [ ]:
sentiment_pipe(texts_v1, **sent_kwargs)

In [ ]:
sentiment_pipe_v2(texts_v2, **sent_kwargs)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Tristan/gpt2_reward_summarization")
model = AutoModelForSequenceClassification.from_pretrained("Tristan/gpt2_reward_summarization")


bad_summary = "Ban Ki-moon was a very good president."
good_summary = 


bad_input_ids = tokenizer.encode("summary:"+ bad_summary + tokenizer.eos_token + "article:" + article)
bad_score = model(input_ids=torch.tensor([bad_input_ids]))[0]
print("bad", bad_score[0].detach())
good_input_ids = tokenizer.encode("summary:" + good_summary + tokenizer.eos_token + "article:" + article)
good_score = model(input_ids=torch.tensor([good_input_ids]))[0]
print("good", good_score[0].detach())

In [ ]:
bad_summary = 

In [ ]:

response_text = good_summary + " " + tokenizer.bos_token + " " + article

In [ ]:
bad_input_ids = tokenizer.encode(bad_summary + " " + tokenizer.bos_token + " " + article)
bad_score = model(input_ids=torch.tensor([bad_input_ids]))[0]
print("bad", bad_score[0].detach())

In [ ]:
good_input_ids = tokenizer.encode(response_text)
good_score = model(input_ids=torch.tensor([good_input_ids]))[0]
print("good", good_score[0].detach())

In [ ]:
def turn_into_text_classification_format(examples):
    new_examples = {"text_j": [], "text_k": []}
    for info, summaries, choice in zip(examples["info"], examples["summaries"], examples["choice"]):
        if len(summaries) != 2 or choice not in (0, 1):
            raise ValueError(
                f"There should be two summaries with a choice that's either 0 or 1. Received {len(summaries)} summaries and choice={choice}."
            )
        original_text_field = "post" if info["post"] is not None else "article"
        new_examples["text_j"].append(
            summaries[choice]["text"] + " " + tokenizer.bos_token + " " + info[original_text_field]
        )
        new_examples["text_k"].append(
            summaries[0 if choice == 1 else 1]["text"] + " " + tokenizer.bos_token + " " + info[original_text_field]
        )

    return new_examples